Please run those two cells before running the Notebook!

As those plotting settings are standard throughout the book, we do not show them in the book every time we plot something.

In [ ]:
# %matplotlib inline
%config InlineBackend.figure_format = "retina"

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

import warnings
from pandas.errors import SettingWithCopyWarning  # ✅ FIXED import

# Ignore future and SettingWithCopy warnings
warnings.simplefilter(action="ignore", category=FutureWarning)
warnings.simplefilter(action="ignore", category=SettingWithCopyWarning)

# Set Seaborn theme for plots
sns.set_theme(context="talk", style="whitegrid", 
              palette="colorblind", color_codes=True, 
              rc={"figure.figsize": [12, 8]})


# Chapter 1 - Acquiring Financial Data 

## 1.1 Getting data from Yahoo Finance

### How to do it...

1. Import the libraries:

In [ ]:
import pandas as pd 
import yfinance as yf

2. Download the data:

In [ ]:
df = yf.download("AAPL", 
                 start="2011-01-01", 
                 end="2021-12-31",
                 progress=False)

3. Inspect the data:

In [ ]:
print(f"Downloaded {len(df)} rows of data.")
df

### There's more

We can also use the `Ticker` class to download the historical prices and much more.

In [ ]:
# instantiate the Ticker class
aapl_data = yf.Ticker("AAPL")

In [ ]:
# get the last month of historical prices
aapl_data.history()

In [ ]:
# get stock's info
aapl_data.info

In [ ]:
# show corporate actions
aapl_data.actions

In [ ]:
# show financials
aapl_data.financials

In [ ]:
# show quarterly financials
aapl_data.quarterly_financials

In [ ]:
# Show Annual Net Income (replaces deprecated .earnings)
aapl_data.income_stmt.loc['Net Income']


In [ ]:
# show quarterly earnings
aapl_data.quarterly_earnings

In [ ]:
# show upcoming events, such as earnings
aapl_data.calendar

## 1.2 Getting data from Nasdaq Data Link 

### How to do it...

1. Import the libraries:

In [ ]:
!pip install nasdaq-data-link

In [ ]:
import pandas as pd 
import nasdaqdatalink

2. Authenticate using the personal API key:

In [ ]:
nasdaqdatalink.ApiConfig.api_key = "YOUR_KEY_HERE"

3. Download the data:

In [ ]:
# nasdaqdatalink.ApiConfig.api_key = "YOUR_KEY_HERE" # This line is also not needed if the section is unused

# df = nasdaqdatalink.get(dataset="WIKI/AAPL",
#                         start_date="2011-01-01", 
#                         end_date="2021-12-31")

# print(f"Downloaded {len(df)} rows of data.")
# df.head()

4. Inspect the data:

In [ ]:
print(f"Downloaded {len(df)} rows of data.")
df.head()

### There's more

1. Download the data for multiple tickers using the `get_table` function:

In [ ]:
COLUMNS = ["ticker", "date", "adj_close"]
df = nasdaqdatalink.get_table("WIKI/PRICES", 
                              ticker=["AAPL", "MSFT", "INTC"], 
                              qopts={"columns": COLUMNS}, 
                              date={"gte": "2011-01-01", 
                                    "lte": "2021-12-31"}, 
                              paginate=True)
df.head()

2. Pivot the data from long to wide:

In [ ]:
# set the date as index
df = df.set_index("date")

# use the pivot function to reshape the data
df_wide = df.pivot(columns="ticker")
df_wide.head()

## 1.3 Getting data from Intrinio

### How to do it...

1. Import the libraries:

In [ ]:
!pip install intrinio-sdk

In [ ]:
import intrinio_sdk as intrinio
import pandas as pd

2. Authenticate using the personal API key and select the API:

In [ ]:
# Replace "YOUR_KEY_HERE" with the key you received
intrinio.ApiClient().set_api_key("OjVkODc5MGI4NGI5MjUwYmM5NmI2ZDhiM2QyZTM5MDFi") 
security_api = intrinio.SecurityApi()

3. Request the data:

In [ ]:
r = security_api.get_security_stock_prices(
    identifier="AAPL", 
    start_date="2011-01-01",
    end_date="2021-12-31", 
    frequency="daily",
    page_size=10000
)

4. Convert the results into a DataFrame:

In [ ]:
df = (
    pd.DataFrame(r.stock_prices_dict)
    .sort_values("date")
    .set_index("date")
)

5. Inspect the data:

In [ ]:
print(f"Downloaded {df.shape[0]} rows of data.")

df.head()

### There's more

1. Get Coca-Cola's real-time stock price:

In [ ]:
security_api.get_security_realtime_price("KO")

2. Download news articles related to Coca-Cola:

In [ ]:
r = intrinio.CompanyApi().get_company_news(
    identifier="KO", 
    page_size=100
)

df = pd.DataFrame(r.news_dict)
df.head()

3. Search for companies connected to the search phrase:

Returns a list of companies recognized by the Thea API in the given `text` query string parameter. 

In [ ]:
r = intrinio.CompanyApi().recognize_company("Intel")
df = pd.DataFrame(r.companies_dict)
df

4. Get Coca-Cola's intraday stock prices:

In [ ]:
response = (
    security_api.get_security_intraday_prices(identifier="KO", 
                                              start_date="2021-01-02",
                                              end_date="2021-01-05",
                                              page_size=1000)
    )
df = pd.DataFrame(response.intraday_prices_dict)
df

5. Get Coca-Cola's latest earnings record:

In [ ]:
r = security_api.get_security_latest_earnings_record(identifier="KO")
print(r)

## 1.4 Getting data from Alpha Vantage

### How to do it...

1. Import the libraries:

In [ ]:
!pip install alpha-vantage

In [ ]:
from alpha_vantage.cryptocurrencies import CryptoCurrencies

2. Authenticate using the personal API key and select the API:

In [ ]:
# Replace "YOUR_KEY_HERE" with your key
ALPHA_VANTAGE_API_KEY = "IE92VT1QH9CXI261"

# This will now work
crypto_api = CryptoCurrencies(key=ALPHA_VANTAGE_API_KEY, 
                              output_format="pandas")

3. Download the daily prices of Bitcoin, expressed in EUR:

In [ ]:
data, meta_data = crypto_api.get_digital_currency_daily(
    symbol="BTC", 
    market="EUR"
)

In [ ]:
meta_data

In [ ]:
data

4. Download the realtime exchange rate:

In [ ]:
crypto_api.get_digital_currency_exchange_rate(
    from_currency="BTC", 
    to_currency="USD"
)[0].transpose()

### There's more

1. Import the libraries:

In [ ]:
import requests
import pandas as pd
from io import BytesIO

2. Download Bitcoin's intraday data:

In [ ]:
AV_API_URL = "https://www.alphavantage.co/query"
parameters = {
    "function": "CRYPTO_INTRADAY",
    "symbol": "ETH",
    "market": "USD",
    "interval": "30min",
    "outputsize": "full",
    "apikey": ALPHA_VANTAGE_API_KEY
}
r = requests.get(AV_API_URL, params=parameters)
data = r.json()
df = (
    pd.DataFrame(data["Time Series Crypto (30min)"])
    .transpose()
)
df

3. Download the upcoming earnings announcements:

In [ ]:
AV_API_URL = "https://www.alphavantage.co/query" 
parameters = {
    "function": "EARNINGS_CALENDAR",
    "horizon": "3month",
    "apikey": ALPHA_VANTAGE_API_KEY
}

r = requests.get(AV_API_URL, params=parameters)
pd.read_csv(BytesIO(r.content))

In [ ]:
type(r.content)

### Bonus

This section includes some additional short recipes not published in the book.

1. Get Tesla's earnings:

In [ ]:
AV_API_URL = "https://www.alphavantage.co/query" 
parameters = {
    "function": "EARNINGS",
    "symbol": "TSLA", 
    "apikey": ALPHA_VANTAGE_API_KEY
}
r = requests.get(AV_API_URL,  params=parameters)
data = r.json()

In [ ]:
data.keys()

In [ ]:
df = pd.DataFrame(data["quarterlyEarnings"])
df.head()

2. Download the upcoming IPOs:

In [ ]:
import csv

AV_API_URL = "https://www.alphavantage.co/query" 
parameters = {
    "function": "IPO_CALENDAR",
    "apikey": ALPHA_VANTAGE_API_KEY
}

with requests.Session() as s:
    download = s.get(AV_API_URL, params=parameters)
    decoded_content = download.content.decode("utf-8")
    ipos_list = list(csv.reader(decoded_content.splitlines(), delimiter=","))

df = pd.DataFrame(ipos_list[1:], columns=ipos_list[0])
df

3. Download Google's stock prices using the `TimeSeries` module:

In [ ]:
from alpha_vantage.timeseries import TimeSeries

ts = TimeSeries(key=ALPHA_VANTAGE_API_KEY, output_format="pandas")
data, _ = ts.get_daily("GOOGL", outputsize="full")
print(f"Downloaded {len(data)} observations")
data

4. Similarly, we can download the intraday prices:

In [ ]:
data, _ = ts.get_intraday("GOOGL", interval="5min", outputsize="full")
data

## 1.5 Getting data from CoinGecko

### How to do it...

1. Import the libraries:

In [ ]:
!pip install pycoingecko

In [ ]:
from pycoingecko import CoinGeckoAPI
from datetime import datetime
import pandas as pd

2. Instantiate the CoinGecko API:

In [ ]:
cg = CoinGeckoAPI()

3. Get Bitcoin's OHLC prices from the last 14 days:

In [ ]:
ohlc = cg.get_coin_ohlc_by_id(
    id="bitcoin", vs_currency="usd", days="14"
)
ohlc_df = pd.DataFrame(ohlc)
ohlc_df.columns = ["date", "open", "high", "low", "close"]
ohlc_df["date"] = pd.to_datetime(ohlc_df["date"], unit="ms")
ohlc_df

### There's more

1. Get the top 7 trending coins (based on the # of searches in the last 24h) from CoinGecko:

In [ ]:
trending_coins = cg.get_search_trending()
(
    pd.DataFrame([coin["item"] for coin in trending_coins["coins"]])
    .drop(columns=["thumb", "small", "large"])
)

2. Get Bitcoin's current price in USD:

In [ ]:
cg.get_price(ids="bitcoin", vs_currencies="usd")

### Bonus

This section includes some additional short recipes not published in the book.

1. Get current price of ETH and BTC in USD and EUR:

In [ ]:
cg.get_price(ids=["bitcoin", "ethereum"], vs_currencies=["usd", "eur"])

2. Get the current BTC/USD eschange rate, market capitalization, 24h volumne and change and the last-updated timestamp:

In [ ]:
cg.get_price(ids="bitcoin", vs_currencies="usd", 
             include_market_cap=True, 
             include_24hr_vol=True, 
             include_24hr_change=True, 
             include_last_updated_at=True)

3. Get the list of all supported coin ids, together with their name and symbol:

In [ ]:
pd.DataFrame(cg.get_coins_list())

4. Get all the coins market data:

In [ ]:
pd.DataFrame(cg.get_coins_markets(vs_currency="eur"))

5. Get all the supported crypto exchanges:

In [ ]:
exchanges_df = pd.DataFrame(cg.get_exchanges_list(per_page=250))
exchanges_df.head()

6. Get a summary of DEFI:

In [ ]:
cg.get_global_decentralized_finance_defi()